In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("titanic_sdf").getOrCreate()
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/27 12:43:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Spark DataFrame
- csv에서 불러오기

In [2]:
filepath = "/home/ubuntu/working/spark-examples/data/titanic_train.csv"
titanic_sdf = spark.read.csv(filepath, inferSchema=True, header=True) # inferSchema : 컬럼 타입 자동 추론
titanic_sdf

DataFrame[PassengerId: int, Survived: int, Pclass: int, Name: string, Sex: string, Age: double, SibSp: int, Parch: int, Ticket: string, Fare: double, Cabin: string, Embarked: string]

In [3]:
# 스파크의 데이터프레임은 정형데이터 구조이기 때문에 스키마가 "반드시 존재"
titanic_sdf.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [4]:
# 데이터 확인 - RDD의 collect() 또는 take()에 대응
titanic_sdf.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

## Pandas DataFrame과 비교

In [5]:
import pandas as pd

titanic_pdf = pd.read_csv(filepath, header='infer')
titanic_pdf.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
# Spark DataFrame의 head() 메소드는 선두 N개의 Row가 들어있는 "리스트"를 반환. DataFrame API 사용 X
titanic_sdf.head(5)

[Row(PassengerId=1, Survived=0, Pclass=3, Name='Braund, Mr. Owen Harris', Sex='male', Age=22.0, SibSp=1, Parch=0, Ticket='A/5 21171', Fare=7.25, Cabin=None, Embarked='S'),
 Row(PassengerId=2, Survived=1, Pclass=1, Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex='female', Age=38.0, SibSp=1, Parch=0, Ticket='PC 17599', Fare=71.2833, Cabin='C85', Embarked='C'),
 Row(PassengerId=3, Survived=1, Pclass=3, Name='Heikkinen, Miss. Laina', Sex='female', Age=26.0, SibSp=0, Parch=0, Ticket='STON/O2. 3101282', Fare=7.925, Cabin=None, Embarked='S'),
 Row(PassengerId=4, Survived=1, Pclass=1, Name='Futrelle, Mrs. Jacques Heath (Lily May Peel)', Sex='female', Age=35.0, SibSp=1, Parch=0, Ticket='113803', Fare=53.1, Cabin='C123', Embarked='S'),
 Row(PassengerId=5, Survived=0, Pclass=3, Name='Allen, Mr. William Henry', Sex='male', Age=35.0, SibSp=0, Parch=0, Ticket='373450', Fare=8.05, Cabin=None, Embarked='S')]

In [7]:
# 스파크에서 판다스 데이터프레임의 head()와 비슷한 역할을 하는 메소드
# limit : Transformations
titanic_sdf.limit(5).show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

In [8]:
print(titanic_pdf)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...   ... 

In [9]:
print(titanic_sdf)

DataFrame[PassengerId: int, Survived: int, Pclass: int, Name: string, Sex: string, Age: double, SibSp: int, Parch: int, Ticket: string, Fare: double, Cabin: string, Embarked: string]


In [10]:
# Spark DataFrame -> Pandas DataFrame
titanic_sdf.select("Pclass", "Embarked").toPandas() # toPandas() : Actions

,Pclass,Embarked
0,3,S
1,1,C
2,3,S
3,1,S
4,3,S
...,...,...
886,2,S
887,1,S
888,3,S
889,1,C


In [11]:
# describe() 메소드를 활용해서 데이터프레임의 통계적인 정보를 확인 할 수 있다
titanic_pdf.describe() # 숫자형 데이터에 대해서만 통계적인 정보를 구한다.

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [12]:
titanic_sdf.describe().show() # 문자열 형태의 컬럼에 대해서도 집계가 된다.

23/07/27 12:43:57 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

In [13]:
# 스파크 데이터프레임에서 숫자 형식의 데이터에 대한 집계 해보기
titanic_sdf.dtypes

[('PassengerId', 'int'),
 ('Survived', 'int'),
 ('Pclass', 'int'),
 ('Name', 'string'),
 ('Sex', 'string'),
 ('Age', 'double'),
 ('SibSp', 'int'),
 ('Parch', 'int'),
 ('Ticket', 'string'),
 ('Fare', 'double'),
 ('Cabin', 'string'),
 ('Embarked', 'string')]

In [14]:
number_columns = [column_name for column_name, dtype in titanic_sdf.dtypes if dtype != 'string']
number_columns

['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

In [20]:
titanic_sdf.select(number_columns).describe().show()

+-------+-----------------+-------------------+------------------+------------------+------------------+-------------------+-----------------+
|summary|      PassengerId|           Survived|            Pclass|               Age|             SibSp|              Parch|             Fare|
+-------+-----------------+-------------------+------------------+------------------+------------------+-------------------+-----------------+
|  count|              891|                891|               891|               714|               891|                891|              891|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642| 29.69911764705882|0.5230078563411896|0.38159371492704824| 32.2042079685746|
| stddev|257.3538420152301|0.48659245426485753|0.8360712409770491|14.526497332334035|1.1027434322934315| 0.8060572211299488|49.69342859718089|
|    min|                1|                  0|                 1|              0.42|                 0|                  0|              0.0|

In [21]:
# shape
# Pandas DataFrame의 shape은 row와 column의 개수가 나온다.
titanic_pdf.shape

(891, 12)

In [23]:
# Spark DataFrame은 shape이 없다.
print(len( titanic_sdf.columns ))

12


In [24]:
print(titanic_sdf.count())

891


# Spark DataFrame의 select() 메소드 알아보기⭐️
- `select` 메소드는 SQL의 SELECT절과 매우 흡사하게 한 개 이상의 컬럼들의 값을 **DATAFRAME** 형식으로 반환
- 한 개의 컬럼명 또는 여러 개의 컬럼명을 인자로 받을 수 있다.
    - 개별 컬럼명을 문자열 형태로 또는 DataFrame의 컬럼 속성으로 지정
- DataFrame의 컬럼 속성으로 지정할 시에는 DataFrame.컬럼명, DataFrame[컬럼명], `col(컬럼명)`으로 지정

In [25]:
# Pandas 데이터프레임에서 단일 컬럼값 가져오기
titanic_pdf['Name']

0                                Braund, Mr. Owen Harris
1      Cumings, Mrs. John Bradley (Florence Briggs Th...
2                                 Heikkinen, Miss. Laina
3           Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                               Allen, Mr. William Henry
                             ...                        
886                                Montvila, Rev. Juozas
887                         Graham, Miss. Margaret Edith
888             Johnston, Miss. Catherine Helen "Carrie"
889                                Behr, Mr. Karl Howell
890                                  Dooley, Mr. Patrick
Name: Name, Length: 891, dtype: object

In [26]:
titanic_pdf[["Name", "Age"]]

,Name,Age
0,"Braund, Mr. Owen Harris",22.0
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0
2,"Heikkinen, Miss. Laina",26.0
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0
4,"Allen, Mr. William Henry",35.0
...,...,...
886,"Montvila, Rev. Juozas",27.0
887,"Graham, Miss. Margaret Edith",19.0
888,"Johnston, Miss. Catherine Helen ""Carrie""",NaN
889,"Behr, Mr. Karl Howell",26.0


In [27]:
# Spark Dataframe의 select 사용. select는 Transformations
titanic_sdf.select("Name").show() # select name from titanic_sdf

+--------------------+
|                Name|
+--------------------+
|Braund, Mr. Owen ...|
|Cumings, Mrs. Joh...|
|Heikkinen, Miss. ...|
|Futrelle, Mrs. Ja...|
|Allen, Mr. Willia...|
|    Moran, Mr. James|
|McCarthy, Mr. Tim...|
|Palsson, Master. ...|
|Johnson, Mrs. Osc...|
|Nasser, Mrs. Nich...|
|Sandstrom, Miss. ...|
|Bonnell, Miss. El...|
|Saundercock, Mr. ...|
|Andersson, Mr. An...|
|Vestrom, Miss. Hu...|
|Hewlett, Mrs. (Ma...|
|Rice, Master. Eugene|
|Williams, Mr. Cha...|
|Vander Planke, Mr...|
|Masselmani, Mrs. ...|
+--------------------+
only showing top 20 rows



In [28]:
# 여러 개의 컬럼 select
titanic_sdf.select("Name", "Fare").show()

+--------------------+-------+
|                Name|   Fare|
+--------------------+-------+
|Braund, Mr. Owen ...|   7.25|
|Cumings, Mrs. Joh...|71.2833|
|Heikkinen, Miss. ...|  7.925|
|Futrelle, Mrs. Ja...|   53.1|
|Allen, Mr. Willia...|   8.05|
|    Moran, Mr. James| 8.4583|
|McCarthy, Mr. Tim...|51.8625|
|Palsson, Master. ...| 21.075|
|Johnson, Mrs. Osc...|11.1333|
|Nasser, Mrs. Nich...|30.0708|
|Sandstrom, Miss. ...|   16.7|
|Bonnell, Miss. El...|  26.55|
|Saundercock, Mr. ...|   8.05|
|Andersson, Mr. An...| 31.275|
|Vestrom, Miss. Hu...| 7.8542|
|Hewlett, Mrs. (Ma...|   16.0|
|Rice, Master. Eugene| 29.125|
|Williams, Mr. Cha...|   13.0|
|Vander Planke, Mr...|   18.0|
|Masselmani, Mrs. ...|  7.225|
+--------------------+-------+
only showing top 20 rows



In [29]:
# List를 이용해서 select
select_columns = ["Name", "Age", "Pclass", "Fare"]

In [30]:
# *list 또는 list를 그대로 사용해서 데이터를 조회
#  *list : list내의 모든 원소를 unpack한다. 스파크 데이터프레임에서는 리스트가 아닌, 컬럼의 목록이 그대로 들어가는 것이 원칙!
# select("A", "B", "C") 처럼 선택하는 것이 원칙(구 버전). 신 버전에서는 select(["A", "B", "C"]) 로 선택하는 것을 지원
print(select_columns)
print(*select_columns)

['Name', 'Age', 'Pclass', 'Fare']
Name Age Pclass Fare


In [31]:
titanic_sdf.select(select_columns).show() # 신 버전
titanic_sdf.select(*select_columns).show() # 구 버전

+--------------------+----+------+-------+
|                Name| Age|Pclass|   Fare|
+--------------------+----+------+-------+
|Braund, Mr. Owen ...|22.0|     3|   7.25|
|Cumings, Mrs. Joh...|38.0|     1|71.2833|
|Heikkinen, Miss. ...|26.0|     3|  7.925|
|Futrelle, Mrs. Ja...|35.0|     1|   53.1|
|Allen, Mr. Willia...|35.0|     3|   8.05|
|    Moran, Mr. James|null|     3| 8.4583|
|McCarthy, Mr. Tim...|54.0|     1|51.8625|
|Palsson, Master. ...| 2.0|     3| 21.075|
|Johnson, Mrs. Osc...|27.0|     3|11.1333|
|Nasser, Mrs. Nich...|14.0|     2|30.0708|
|Sandstrom, Miss. ...| 4.0|     3|   16.7|
|Bonnell, Miss. El...|58.0|     1|  26.55|
|Saundercock, Mr. ...|20.0|     3|   8.05|
|Andersson, Mr. An...|39.0|     3| 31.275|
|Vestrom, Miss. Hu...|14.0|     3| 7.8542|
|Hewlett, Mrs. (Ma...|55.0|     2|   16.0|
|Rice, Master. Eugene| 2.0|     3| 29.125|
|Williams, Mr. Cha...|null|     2|   13.0|
|Vander Planke, Mr...|31.0|     3|   18.0|
|Masselmani, Mrs. ...|null|     3|  7.225|
+----------

In [32]:
# 컬럼 속성 지정. Pandas 데이터프레임에서 시리즈를 선택하듯이 작성
titanic_sdf['Name']

Column<'Name'>

In [33]:
titanic_pdf["Name"]

0                                Braund, Mr. Owen Harris
1      Cumings, Mrs. John Bradley (Florence Briggs Th...
2                                 Heikkinen, Miss. Laina
3           Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                               Allen, Mr. William Henry
                             ...                        
886                                Montvila, Rev. Juozas
887                         Graham, Miss. Margaret Edith
888             Johnston, Miss. Catherine Helen "Carrie"
889                                Behr, Mr. Karl Howell
890                                  Dooley, Mr. Patrick
Name: Name, Length: 891, dtype: object

# 컬럼 속성 사용하기

In [34]:
# 브라켓([]) 사용하기. 이 방법은 많이 쓰이는 방법은 아님!
titanic_sdf.select(titanic_sdf['Name']).show()

+--------------------+
|                Name|
+--------------------+
|Braund, Mr. Owen ...|
|Cumings, Mrs. Joh...|
|Heikkinen, Miss. ...|
|Futrelle, Mrs. Ja...|
|Allen, Mr. Willia...|
|    Moran, Mr. James|
|McCarthy, Mr. Tim...|
|Palsson, Master. ...|
|Johnson, Mrs. Osc...|
|Nasser, Mrs. Nich...|
|Sandstrom, Miss. ...|
|Bonnell, Miss. El...|
|Saundercock, Mr. ...|
|Andersson, Mr. An...|
|Vestrom, Miss. Hu...|
|Hewlett, Mrs. (Ma...|
|Rice, Master. Eugene|
|Williams, Mr. Cha...|
|Vander Planke, Mr...|
|Masselmani, Mrs. ...|
+--------------------+
only showing top 20 rows



In [35]:
titanic_sdf['Fare'] * 100

Column<'(Fare * 100)'>

In [36]:
titanic_sdf.select(
    titanic_sdf["Fare"],
    titanic_sdf["Fare"] * 100
).show()

+-------+-----------------+
|   Fare|     (Fare * 100)|
+-------+-----------------+
|   7.25|            725.0|
|71.2833|          7128.33|
|  7.925|            792.5|
|   53.1|           5310.0|
|   8.05|805.0000000000001|
| 8.4583|845.8299999999999|
|51.8625|          5186.25|
| 21.075|           2107.5|
|11.1333|          1113.33|
|30.0708|          3007.08|
|   16.7|           1670.0|
|  26.55|           2655.0|
|   8.05|805.0000000000001|
| 31.275|           3127.5|
| 7.8542|           785.42|
|   16.0|           1600.0|
| 29.125|           2912.5|
|   13.0|           1300.0|
|   18.0|           1800.0|
|  7.225|            722.5|
+-------+-----------------+
only showing top 20 rows



스파크 데이터프레임에서 컬럼을 다룰 때 가장 많이 사용하는 방식
- `col`
- Spark SQL에서 사용되는 모든 연산이 들어있는 패키지인 `pyspark.sql.functions` 에서 import

In [38]:
import pyspark.sql.functions as F

titanic_sdf.select(
    F.col("Name"),
    F.col("Fare")
).show()

+--------------------+-------+
|                Name|   Fare|
+--------------------+-------+
|Braund, Mr. Owen ...|   7.25|
|Cumings, Mrs. Joh...|71.2833|
|Heikkinen, Miss. ...|  7.925|
|Futrelle, Mrs. Ja...|   53.1|
|Allen, Mr. Willia...|   8.05|
|    Moran, Mr. James| 8.4583|
|McCarthy, Mr. Tim...|51.8625|
|Palsson, Master. ...| 21.075|
|Johnson, Mrs. Osc...|11.1333|
|Nasser, Mrs. Nich...|30.0708|
|Sandstrom, Miss. ...|   16.7|
|Bonnell, Miss. El...|  26.55|
|Saundercock, Mr. ...|   8.05|
|Andersson, Mr. An...| 31.275|
|Vestrom, Miss. Hu...| 7.8542|
|Hewlett, Mrs. (Ma...|   16.0|
|Rice, Master. Eugene| 29.125|
|Williams, Mr. Cha...|   13.0|
|Vander Planke, Mr...|   18.0|
|Masselmani, Mrs. ...|  7.225|
+--------------------+-------+
only showing top 20 rows



In [40]:
F.col("Name") * 100

Column<'(Name * 100)'>

In [42]:
titanic_sdf.select(
    F.col("Name"),
    F.upper(F.col("Name")).alias("Cap Name")
).show()

+--------------------+--------------------+
|                Name|            Cap Name|
+--------------------+--------------------+
|Braund, Mr. Owen ...|BRAUND, MR. OWEN ...|
|Cumings, Mrs. Joh...|CUMINGS, MRS. JOH...|
|Heikkinen, Miss. ...|HEIKKINEN, MISS. ...|
|Futrelle, Mrs. Ja...|FUTRELLE, MRS. JA...|
|Allen, Mr. Willia...|ALLEN, MR. WILLIA...|
|    Moran, Mr. James|    MORAN, MR. JAMES|
|McCarthy, Mr. Tim...|MCCARTHY, MR. TIM...|
|Palsson, Master. ...|PALSSON, MASTER. ...|
|Johnson, Mrs. Osc...|JOHNSON, MRS. OSC...|
|Nasser, Mrs. Nich...|NASSER, MRS. NICH...|
|Sandstrom, Miss. ...|SANDSTROM, MISS. ...|
|Bonnell, Miss. El...|BONNELL, MISS. EL...|
|Saundercock, Mr. ...|SAUNDERCOCK, MR. ...|
|Andersson, Mr. An...|ANDERSSON, MR. AN...|
|Vestrom, Miss. Hu...|VESTROM, MISS. HU...|
|Hewlett, Mrs. (Ma...|HEWLETT, MRS. (MA...|
|Rice, Master. Eugene|RICE, MASTER. EUGENE|
|Williams, Mr. Cha...|WILLIAMS, MR. CHA...|
|Vander Planke, Mr...|VANDER PLANKE, MR...|
|Masselmani, Mrs. ...|MASSELMANI

In [43]:
# .을 이용해서도 컬럼 속성을 지정할 수 있음. 이 방식도 자주 사용되는 방식은 아님

titanic_sdf.select(
    titanic_sdf.Name,
    titanic_sdf.Fare * 100,
    F.lower(titanic_sdf.Embarked).alias("lower embarked")
).show()

+--------------------+-----------------+--------------+
|                Name|     (Fare * 100)|lower embarked|
+--------------------+-----------------+--------------+
|Braund, Mr. Owen ...|            725.0|             s|
|Cumings, Mrs. Joh...|          7128.33|             c|
|Heikkinen, Miss. ...|            792.5|             s|
|Futrelle, Mrs. Ja...|           5310.0|             s|
|Allen, Mr. Willia...|805.0000000000001|             s|
|    Moran, Mr. James|845.8299999999999|             q|
|McCarthy, Mr. Tim...|          5186.25|             s|
|Palsson, Master. ...|           2107.5|             s|
|Johnson, Mrs. Osc...|          1113.33|             s|
|Nasser, Mrs. Nich...|          3007.08|             c|
|Sandstrom, Miss. ...|           1670.0|             s|
|Bonnell, Miss. El...|           2655.0|             s|
|Saundercock, Mr. ...|805.0000000000001|             s|
|Andersson, Mr. An...|           3127.5|             s|
|Vestrom, Miss. Hu...|           785.42|        

# Spark DataFrame의 filter() 메소드 알아보기
- `filter()`는 SQL의 WHERE과 매우 흡사하게 특정 조건을 만족하는 레코드를 DataFrame으로 반환.
- `filter()`내의 조건 컬럼은 컬럼 속성으로 지정이 가능.
- 조건문 자체는 SQL과 유사한 문자열로 지정할 수 있다.
    - 단, 조건 컬럼은 문자열 지정이 안된다.
- `where` 메소드는 filter의 alias 버전이며, where과 직관적인 동일성을 간주하기 위해 생성.
- 복합 조건 and는 &를, or를 사용. 개별 조건은 괄호()로 감싸야 한다.


In [46]:
"Embarked" == "Q"

False

In [50]:
# titanic_sdf.filter("Embarked==Q").show()
# titanic_sdf.filter("Embarked"=="Q").show()

titanic_sdf.filter(F.col("Embarked") == "Q").show()

+-----------+--------+------+--------------------+------+----+-----+-----+------+------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|Ticket|  Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+------+------+-----+--------+
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|330877|8.4583| null|       Q|
|         17|       0|     3|Rice, Master. Eugene|  male| 2.0|    4|    1|382652|29.125| null|       Q|
|         23|       1|     3|"McGowan, Miss. A...|female|15.0|    0|    0|330923|8.0292| null|       Q|
|         29|       1|     3|"O'Dwyer, Miss. E...|female|null|    0|    0|330959|7.8792| null|       Q|
|         33|       1|     3|Glynn, Miss. Mary...|female|null|    0|    0|335677|  7.75| null|       Q|
|         45|       1|     3|Devaney, Miss. Ma...|female|19.0|    0|    0|330958|7.8792| null|       Q|
|         47|       0|     3|   Lennon, Mr. Denis|  male|null|  

In [51]:
titanic_sdf.where(F.col("Embarked") == "S").show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          7|       0|     1|McCarthy, Mr. Tim...|  male|54.0|    0|    0|           17463|51.8625|  E46|       S|
|          8|       0|     3|Palsson, Master. ...|  male| 2.0|    3|    1|      

In [53]:
titanic_sdf.filter("Embarked == 'Q'").show()

+-----------+--------+------+--------------------+------+----+-----+-----+------+------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|Ticket|  Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+------+------+-----+--------+
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|330877|8.4583| null|       Q|
|         17|       0|     3|Rice, Master. Eugene|  male| 2.0|    4|    1|382652|29.125| null|       Q|
|         23|       1|     3|"McGowan, Miss. A...|female|15.0|    0|    0|330923|8.0292| null|       Q|
|         29|       1|     3|"O'Dwyer, Miss. E...|female|null|    0|    0|330959|7.8792| null|       Q|
|         33|       1|     3|Glynn, Miss. Mary...|female|null|    0|    0|335677|  7.75| null|       Q|
|         45|       1|     3|Devaney, Miss. Ma...|female|19.0|    0|    0|330958|7.8792| null|       Q|
|         47|       0|     3|   Lennon, Mr. Denis|  male|null|  

In [54]:
# AND -> &
# Embakred가 S고, Pclass가 1인 사람의 정보
titanic_sdf.filter(
    ( F.col("Embarked") == 'S' ) & ( F.col("Pclass") == 1)
).show()

+-----------+--------+------+--------------------+------+----+-----+-----+-----------+-------+-----------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|     Ticket|   Fare|      Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+-----------+-------+-----------+--------+
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|     113803|   53.1|       C123|       S|
|          7|       0|     1|McCarthy, Mr. Tim...|  male|54.0|    0|    0|      17463|51.8625|        E46|       S|
|         12|       1|     1|Bonnell, Miss. El...|female|58.0|    0|    0|     113783|  26.55|       C103|       S|
|         24|       1|     1|Sloper, Mr. Willi...|  male|28.0|    0|    0|     113788|   35.5|         A6|       S|
|         28|       0|     1|Fortune, Mr. Char...|  male|19.0|    3|    2|      19950|  263.0|C23 C25 C27|       S|
|         36|       0|     1|Holverson, Mr. Al...|  male|42.0|    1|    

In [57]:
# OR -> |
# Embarked가 S이거나, Pclass가 2인 사람의 정보
titanic_sdf.filter(
    (F.col("Embarked") == "S") | (F.col("Pclass") == 2)
).show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          7|       0|     1|McCarthy, Mr. Tim...|  male|54.0|    0|    0|           17463|51.8625|  E46|       S|
|          8|       0|     3|Palsson, Master. ...|  male| 2.0|    3|    1|      

In [58]:
# 이름에 Miss가 들어가는 사람의 정보 가져오기 -> like
titanic_sdf.filter(
    F.col("Name").like("%Miss%")
).show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|         11|       1|     3|Sandstrom, Miss. ...|female| 4.0|    1|    1|         PP 9549|   16.7|   G6|       S|
|         12|       1|     1|Bonnell, Miss. El...|female|58.0|    0|    0|          113783|  26.55| C103|       S|
|         15|       0|     3|Vestrom, Miss. Hu...|female|14.0|    0|    0|          350406| 7.8542| null|       S|
|         23|       1|     3|"McGowan, Miss. A...|female|15.0|    0|    0|          330923| 8.0292| null|       Q|
|         25|       0|     3|Palsson, Miss. To...|female| 8.0|    3|    1|      

In [63]:
# 소문자 h로 이름이 시작되는 사람들의 이름을 조회하고, 이 이름을 대문자로 표현하세요.
# 소문자화 : F.lower, 대문자화 : F.upper

titanic_sdf.filter(
    F.lower(F.col("Name")).like('h%')
).select(
    F.upper(F.col("Name"))
).show()

+--------------------+
|         upper(Name)|
+--------------------+
|HEIKKINEN, MISS. ...|
|HEWLETT, MRS. (MA...|
|HOLVERSON, MR. AL...|
|HARPER, MRS. HENR...|
|HARRIS, MR. HENRY...|
|HOOD, MR. AMBROSE JR|
|HICKMAN, MR. STAN...|
|HAKKARAINEN, MRS....|
|  HALE, MR. REGINALD|
|HONKANEN, MISS. E...|
|  HARRIS, MR. WALTER|
|HOYT, MR. FREDERI...|
|HARRIS, MRS. HENR...|
|HARKNETT, MISS. A...|
|   HOLD, MR. STEPHEN|
|HUNT, MR. GEORGE ...|
|HAMALAINEN, MRS. ...|
|HARRISON, MR. WIL...|
|  HENRY, MISS. DELIA|
|HOSONO, MR. MASABUMI|
+--------------------+
only showing top 20 rows



In [64]:
titanic_sdf.filter(
    "Name like '%Mr%'"
).show()

+-----------+--------+------+--------------------+------+----+-----+-----+---------+-------+-----------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|   Ticket|   Fare|      Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+---------+-------+-----------+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|A/5 21171|   7.25|       null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0| PC 17599|71.2833|        C85|       C|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|   113803|   53.1|       C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|   373450|   8.05|       null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|   330877| 8.4583|       null|       Q|
|          7|       0|     1|McCarthy, Mr. Tim...|  male|54.0|    0|    0|    17463|51.8

In [65]:
titanic_sdf.filter(
    "lower(Name) like '%mr%'"
).show()

+-----------+--------+------+--------------------+------+----+-----+-----+---------+-------+-----------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|   Ticket|   Fare|      Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+---------+-------+-----------+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|A/5 21171|   7.25|       null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0| PC 17599|71.2833|        C85|       C|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|   113803|   53.1|       C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|   373450|   8.05|       null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|   330877| 8.4583|       null|       Q|
|          7|       0|     1|McCarthy, Mr. Tim...|  male|54.0|    0|    0|    17463|51.8

In [15]:
spark.stop()